In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [2]:
datagen_train = ImageDataGenerator(rescale = 1./255)
datagen_test = ImageDataGenerator(rescale = 1./255)

train_dgen = datagen_train.flow_from_directory('../input/fer2013/train',
                                              target_size = (48, 48),
                                              color_mode = "grayscale",
                                              class_mode = "categorical",
                                              batch_size = 64,
                                              shuffle = True)

test_dgen = datagen_test.flow_from_directory('../input/fer2013/test',
                                              target_size = (48, 48),
                                              color_mode = "grayscale",
                                              class_mode = "categorical",
                                              batch_size = 64,
                                              shuffle = False)

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3, 3), padding = "same", input_shape = (48, 48, 1), activation = "relu"))

model.add(Conv2D(64, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(7, activation = "softmax"))


model.summary()

In [5]:
# opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy',
    metrics =['accuracy']
)

In [6]:
history = model.fit_generator(generator = train_dgen,
                             steps_per_epoch = train_dgen.n//train_dgen.batch_size,
                             epochs = 10,
                             validation_data = test_dgen,
                             validation_steps = test_dgen.n//test_dgen.batch_size)

In [7]:
model.save("model.h5")